# One-way Latency Measurements (OWL)

In order to run the One-way latency tool, we recommend the following

1. Create a test slice and prepare the test environment on each node. For these experiments, nodes must be placed on PTP-capable sites and PTP-clock must be enabled.
2. Run OWL between nodes using the OWL docker image..
3. Download the pcap files for data analysis.

Each packet captuer will have two timestamps:  

1) sent timestamp generated by the owl application on the sender side that is inserted into the data load of the UDP packet.
2) received timestamp generated by tcpdump

Latency will be calculated as 2 (received timesamp) - 1 (sent timestamp)

# Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from mflib import owl as owl

try:
    fm = fablib_manager()
                     
    fm.show_config()
except Exception as e:
    print(f"Exception: {e}")

# Create a test slice and set up the test environment on each node

## Choose sites

Since latency measurements require highly-synchronized PTP clocks running on experimental nodes, we need to choose only those sites that are "PTP capable".

### (Optional) Find ALL PTP-capable sites

If you want to choose sites from the list, the following cell will print all the PTP-capable sites currently avaialable 

In [ ]:
sites = fm.get_site_names()
print(f"all FABRIC sites: {len(sites)} ({sites})")

ptp_sites = fm.list_sites(
    output="list",
    quiet=True,
    filter_function = lambda x:x['ptp_capable'] is True and x['state'] == 'Active',
)
ptp_site_names = [x['name'] for x in ptp_sites]

print(f"PTP-enabled and active sites: {len(ptp_site_names)} ({ptp_site_names})")


### Choose 2 PTP-capable sites

In [ ]:
ptp_site_names = fm.get_random_sites(count=2, filter_function=lambda x: x['ptp_capable'] is True and x['state'] == 'Active') 

In [ ]:
print(f"Selected PTP-enabled and active sites: {len(ptp_site_names)} ({ptp_site_names})")

## Create a slice with one node placed on each PTP-enabled site

The following creates a slice with one node on each PTP-capable site, with basic NICs connected to FABRIC's FABnetv4 internet. 

In [ ]:
slice_name = "owl-slice"

In [ ]:
slice = fm.new_slice(name=slice_name)

for i, site_name in enumerate(ptp_site_names):
    node = slice.add_node(name=f"node{i}", site=site_name, image='docker_rocky_8')
    node.add_fabnet()
    print(f"adding node{i} at {site_name}")

#Submit Slice Request
slice.submit();

## (Optional) Observe the Slice's Attributes

In [ ]:
try:
    slice = fm.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

# Prepare each node for time precision experiments (do not edit this section)

In [ ]:
nodes = slice.get_nodes()

## Install and setup linuxptp package  on nodes
Download the Ansible role to configure and install the LinuxPTP software. For more details regarding the steps performed in the playbook, please refer to the repo at [https://github.com/fabric-testbed/ptp](https://github.com/fabric-testbed/ptp)

In [ ]:
pre_requisites = None

# Set Deployment tool repository details
repo_branch = 'main'
repo_name = 'ptp'
destination_folder = f"""/tmp/{repo_name}-{repo_branch}"""
clone_instructions = f"""
cd /tmp/;rm -rf /tmp/{repo_name}-{repo_branch};git clone --branch {repo_branch} https://github.com/fabric-testbed/{repo_name}.git {destination_folder};
"""

### Setting PTP Install Restrictions (Leave it blank)

* If you do not want all interfaces synchronized to PTP, add the name of interfaces to avoid as shown
* Management interfaces are not considered and are avoided by default
* If you do not want the system clock synchronized to PTP set the 'SYNC_SYSTEM_CLOCK' to False
* If you do not have any restrictions for a node, you can omit that node from the list

Example:
```
NODE_RESTRICTIONS = { 
   'node1' : { 'AVOID_IFACES': ['enp6s0'],'SYNC_SYSTEM_CLOCK': False},
   'node2' : { 'AVOID_IFACES': ['enp6s0','enp7s0']},
}
```

In [ ]:
NODE_RESTRICTIONS = {}

### Restrict Ansible operation based on tags (Leave it blank!!!)

* Possible values are ptp_stop,ptp_start,ptp_install 
* Only one tag is allowed
* If empty then all three are performed in the right sequence
* If NODE_RESTRICTIONS are applied along with the tags, the operations will not be performed on the AVOIDED INTERFACES

Example
```
ansible_tags = 'ptp_stop'
```

In [ ]:
ansible_tags = ''

### Run Ansible playbook on each node (Leave it exactly as it is!)

In [ ]:
# Instruction to run ansible command from the node
ansible_instructions = f"""
cd {destination_folder}/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml"""

#Create execute threads
execute_threads = {}

for node in nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    node_name = node.get_name()
    
    # Create JSON files for extra params that will be provided to ansible
    if node_name in NODE_RESTRICTIONS.keys():    
        extra_ansible_params = f""" --extra-vars @parameters.json""";
        with open('/tmp/'+node_name+'-parameters.json', 'w') as f:
            json.dump(NODE_RESTRICTIONS[node_name], f)
        print (f"Uploading install restrictions for {node_name}")    
        node.upload_file('/tmp/'+node_name+'-parameters.json',destination_folder+'/ansible/parameters.json')
    else:
        extra_ansible_params = ''
    if ansible_tags != '':
        extra_ansible_params = extra_ansible_params + ' --tags '+ansible_tags
        
    print (f"Running the PTP Deployment Ansible Playbook on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}"\
                f"{extra_ansible_params}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()

print (f"Ansible Playbook run on all nodes completed\n")

## Start Docker on each node and verify it is running

`docker ps` should print `CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES`

In [ ]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker ps")

# Run OWL

Now all the set up has been done, time to run OWL!

## Enter the docker image name

Image will be pulled from Docker Hub by default. The OWL docker images are found [here](https://hub.docker.com/r/fabrictestbed/owl)

In [ ]:
# Change the version number as necessary
image_name="fabrictestbed/owl:0.2.0"  

#### (Optional) Pull Docker image on each exp node

Docker start (in the cells below) should pull the image automatically.
But if pulling the image is necessary, use these lines.

In [ ]:
# for node in nodes:
#     print(node.get_name())
#     node.execute(f"sudo docker pull {image_name}") 

## Option 1: Use start_owl_all to start sender and capturer on each node

In [ ]:
owl.start_owl_all(slice, img_name=image_name, probe_freq=10, duration=600, delete_previous=True)

### Check the OWL status on all nodes

In [ ]:
owl.check_owl_all(slice)

### Check if pcap files are being generated

In [ ]:
for node in nodes:
    node.execute("hostname")
    node.execute("ls -lh ~/owl-output/")

### Stop OWL containers on all nodes

In [ ]:
owl.stop_owl_all(slice)

## Option 2: Manually start sender and capturer on specific node(s)

In [ ]:
# Start capturer on node0
owl.start_owl_capturer(slice, dst_node=slice.get_node(name="node0"), img_name=image_name, duration=60)

In [ ]:
# Start sender on node1 to send probe packets to node0
owl.start_owl_sender(slice, src_node=slice.get_node(name="node1"), dst_node=slice.get_node(name="node0"), img_name=image_name, duration=60)

### Check the status of containers

In [ ]:
owl.check_owl_all(slice)

### Check if pcap files are being generated

In [ ]:
for node in nodes:
    node.execute("ls -lh ~/owl-output/")

### Stop each capturer and sender containers

In [ ]:
# Stop sender
owl.stop_owl_sender(slice, src_node=slice.get_node(name="node1"), dst_node=slice.get_node(name="node0"))

In [ ]:
# Stop capturer
owl.stop_owl_capturer(slice, dst_node=slice.get_node(name="node0"))

# (Optional) Download pcap files to the current directory

In [ ]:
for node in nodes:
    pcap_name, _ = node.execute("ls ~/owl-output")
    node.download_file(f"./{pcap_name.strip()}", f"/home/rocky/owl-output/{pcap_name.strip()}",)

# Delete the Slice

In [ ]:
# try:
#     slice = fm.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")